In [29]:
!pip install -q konlpy tqdm tensorflow

In [30]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import zipfile

In [31]:
# KoNLPy 사용을 위한 JAVA 환경 변수 설정
def set_java_environment():
    # KoNLPy에서 JAVA 실행 환경 필요
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
    os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
    return

In [32]:
# zip 파일 압축 해제 함수 (구글 드라이브에 zip파일로 상품,리뷰csv파일 올렸렸음)
def unzip_file(zip_path, extract_path):
    import zipfile
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"압축 해제 완료: {extract_path}")

In [33]:
# 데이터 로딩 및 전처리
def load_and_filter_reviews(csv_path):
    df = pd.read_csv(csv_path)

    # 사용할 주요 컬럼만 선택
    df = df[["category","바코드","상품분류코드","상품명", "리뷰어", "평점", "옵션", "사용자 피부 정보","본문"]]

    # 1. 립메이크업 + 아이메이크업 필터링
    df_1 = df[df["category"].isin(["립메이크업", "아이메이크업"])].reset_index(drop=True)

    # 2. 베이스메이크업 중 '블러셔' 관련 키워드 포함 제품 필터링
    keywords = ["블러쉬", "블러시", "블러셔", "치크", "치크밤", "립앤치크"]
    pattern = "|".join(keywords)
    df_2 = df[
        (df["category"] == "베이스메이크업") &
        (df["상품명"].str.contains(pattern, case=False, na=False))
    ].reset_index(drop=True)

    # 두 개의 데이터프레임을 병합
    df = pd.concat([df_1, df_2], ignore_index=True)

    print(f"필터링된 리뷰 데이터 수: {len(df)}")
    return df

In [34]:
# 사용자 피부 정보 컬럼 분해
def split_skin_info(df):
    df_split = df["사용자 피부 정보"].str.split(",", expand=True) # 최대 4개
    df_split.columns = [f"피부_{i+1}" for i in range(df_split.shape[1])]
    return pd.concat([df, df_split], axis=1)

In [35]:
# 퍼스널 컬러 키워드별 컬럼 생성
def create_personal_color_columns(df):
    skin_tones = ["쿨톤", "웜톤", "봄웜톤", "여름쿨톤", "가을웜톤", "겨울쿨톤"]
    for tone in skin_tones:
        df[tone] = df[["피부_1", "피부_2", "피부_3", "피부_4"]].apply(
            lambda row: sum(tone == str(val).strip() for val in row if pd.notna(val)), axis=1
        )
    return df

In [36]:
# 리뷰 본문 전처리 함수 정의
def preprocess_text(text):
    text = str(text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[\u200b\u200c\u200d\ufeff]', '', text)
    text = re.sub(r'[^\w\s가-힣❤️💕✨💖💗💘💓💞💝💜💙💚💛💬💥👍❗️👎☺️😂🤣😭😍😘]', '', text)
    text = re.sub(r'(\w)\1{2,}', r'\1\1', text)
    return text.strip()

In [37]:
# 본문 전처리 컬럼럼을 데이터프레임에 추가
def add_clean_text_column(df):
    df = df[df['본문'].notnull()].reset_index(drop=True)
    df["본문_전처리"] = df["본문"].apply(preprocess_text)
    return df

In [38]:
# 퍼스널 컬러 요약 함수
def summarize_personal_color(df):
    personal_color_cols = ['쿨톤', '웜톤', '봄웜톤', '여름쿨톤', '가을웜톤', '겨울쿨톤']

    def get_personal_color(row): # 1인 칼럼을 가지고 와서 퍼스널컬러 칼럼에 넣음.
        colors = [col for col in personal_color_cols if row[col] == 1]
        if len(colors) == 1:
            return colors[0]
        elif len(colors) > 1:
            return ','.join(colors)
        else:
            return None

    df_new = pd.DataFrame({  # 새로운 DataFrame
        '상품명': df['상품명'],
        '옵션': df['옵션'],
        '퍼스널 컬러': df.apply(get_personal_color, axis=1),
        '리뷰내용': df['본문_전처리']
    })
    return df_new

In [39]:
# 상품명 + 옵션별 리뷰 수 집계
def aggregate_option_reviews(df):
    return (
        df.groupby(['상품명', '옵션'])
        .size()
        .reset_index(name='리뷰수')
        .sort_values(by='리뷰수', ascending=False)
    )

In [40]:
# 제품 정보 병합 및 카테고리 매핑
def merge_product_info(df_reviews, df_product_info):
    category_map = {
        "0113": "스킨/토너", "0114": "에센스/세럼/앰플", "0115": "크림",
        "0116": "로션", "0110": "미스트/오일", "0117": "스킨케어세트", "0118": "스킨케어 디바이스",
        "1001": "클렌징폼/젤", "1004": "오일/밤", "1005": "워터/밀크", "1007": "필링&스크럽",
        "1008": "티슈/패드", "1006": "립&아이리무버", "1009": "클렌징 디바이스",
        "0901": "시트팩", "0904": "패드", "0902": "페이셜팩", "0905": "코팩", "0906": "패치",
        "0206": "립메이크업", "0201": "베이스메이크업", "0207": "아이메이크업",
        "1106": "선크림", "1103": "선스틱", "1104": "선쿠션", "1105": "선스프레이/선패치", "1102": "태닝/애프터선"
    }

    df_merged = df_reviews.merge(
        df_product_info[["상품명", "바코드", "상품분류코드", '비건품목']],
        on="상품명",
        how="left"
    )
    df_merged['품목'] = df_merged['상품분류코드'].astype(str).map(category_map)
    return df_merged


In [43]:
def run_pipeline():
    # 경로 설정
    zip_review_path = '/content/drive/MyDrive/한경x토스/프로젝트/최종 프로젝트/[최종]메이크업_바코드_리뷰_전처리.zip'
    zip_product_path = '/content/drive/MyDrive/한경x토스/프로젝트/최종 프로젝트/[최종]메이크업_바코드_상품_전처리.zip'
    extract_path = '/content/skin_data'

    # 압축 해제
    unzip_file(zip_review_path, extract_path)
    unzip_file(zip_product_path, extract_path)

    ### 1. 립/아이/블러셔용 전처리 (퍼스널컬러 중심 분석)
    df = load_and_filter_reviews(f"{extract_path}/[최종]메이크업_바코드_리뷰_전처리.csv")
    df = split_skin_info(df)
    df = create_personal_color_columns(df)
    df = add_clean_text_column(df)
    df_new = summarize_personal_color(df)

    # 퍼스널컬러 결측치 제거
    df_new = df_new[df_new['퍼스널 컬러'].notna()].reset_index(drop=True)

    option_counts_df = aggregate_option_reviews(df_new)
    print("립/아이/블러셔 리뷰 수 집계:")
    print(option_counts_df)

    df_new.to_csv("메이크업_립&아이&블러셔_옵션과_퍼스널컬러별_요약.csv", encoding="utf-8-sig", index=False)

    ### 2. 제품 정보 불러오기 및 병합
    df_products = pd.read_csv(f"{extract_path}/[최종]메이크업_바코드_상품_전처리.csv", dtype={"상품분류코드": str})
    df_final = merge_product_info(df_new, df_products)
    df_final.to_csv("메이크업(립&아이&블러셔)_RAG_데이터.csv", encoding="utf-8-sig", index=False)

    print("전체 파이프라인 실행 및 모든 데이터 저장 완료")


In [44]:
# 실행 (구글 드라이브 연결 하고 시작)
set_java_environment()
run_pipeline()

압축 해제 완료: /content/skin_data
압축 해제 완료: /content/skin_data


<ipython-input-33-2212522093>:3: DtypeWarning: Columns (8,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


필터링된 리뷰 데이터 수: 648700
립/아이/블러셔 리뷰 수 집계:
                                         상품명                옵션  리뷰수
5694                컬러그램 올인원 오버립 메이커 5 COLOR            02 쿨핑크  948
6402                    헉슬리 립밤 리브 비하인드 3g 6종     핀치 오브 스파이스 3g  720
3951     [튼입술/찐보습]후르디아 블루베리 하이드레이팅 허니 립밤10ml   후르디아 블루베리 허니 립밤  700
3251                       [비건인증]히말라야 립밤 10g    [히말라야] 오리지널 립밤  558
2919   [립마스크/립밤/스틱] 에뛰드 진저슈가 오버나이트 립마스크 외 3종     (단독)립 마스크 15g  549
...                                      ...               ...  ...
196                                  MAC 립스틱               모란지    1
6139                   페리페라 워터 베어 틴트 3.7g 9종     [NEW] 17 로지붐업    1
6138                   페리페라 워터 베어 틴트 3.7g 9종     [NEW] 15 핑쿨거래    1
210                                  MAC 립스틱             캔디 얌얌    1
1977  [NEW출시] 데이지크 츄잉 글로우팟 20종 중 택 1 (단품/기획)    [단품] 06 구아바 캔디    1

[6511 rows x 3 columns]
전체 파이프라인 실행 및 모든 데이터 저장 완료
